## Text Analysis Tutorial

Hello there - we'll be following this Jupyter Notebook for the tutorial. 
The purpose of this tutorial is to walk you through different parts of the text analysis pipeline, from getting a hold of our data, cleaning and annotating it all the way to swapping verbs in sentences and evaluating topic models.

We will not be looking to explore our textual data in depth, but rather in breadth; give a taste of the different kinds of analysis we can do.

Our step, naturally, is setting up our imports. We will be using spaCy for data pre-processing and computational linguistics, gensim for topic modelling, scikit-learn for classification, and Keras for text generation.
We will also use numpy and matplotlib for other parts of the tutorial.

### Imports

In [1]:
import gensim
import numpy as np
import spacy
from spacy import displacy
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import matplotlib.pyplot as plt
import sklearn
import keras

Using Theano backend.


In [2]:
import warnings
import os
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

## Gathering Data

A huge part of text analysis is your data collection - one of the initial goals of the tutorial was to walk the user through the process of cleaning messy twitter data, or scraping data off the internet. But while this does remain an integral part of text analysis, a one and half hour tutorial cannot do justice to both the process of data collection and data analysis - so we will use two more popular, already available data-sets for the purpose of the tutorial.

Keep in mind the only main difference between using a standardised data-set and scraping your own data off the internet is that internet data is largely unstructured; this means we will be spending a lot of time in organising our data into a form that is easy to pre-processes. The datasets we will be working with will be the Lee corpus which is a shortened version of the [Lee Background Corpus](http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF), and the [20NG dataset](http://qwone.com/~jason/20Newsgroups/). We will be performing different tasks with these two datasets, and will talk a little bit more about the datasets when we come across them.

Let us now get started with loading our first data-set, the Lee corpus, which we load using Gensim.

In [3]:
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
text = open(lee_train_file).read()

## Cleaning Data

It's been often said in Machine Learning and NLP algorithms - garbage in, garbage out. We can't have state-of-the-art results without data which is as good. Let's spend this section working on cleaning and understanding our data set.
NTLK is usually a popular choice for pre-processing - but is a rather [outdated](https://explosion.ai/blog/dead-code-should-be-buried) and we will be checking out spaCy, an industry grade text-processing package. 

spaCy uses language models similar to the one we just downloaded before starting this tutorial.

In [4]:
nlp = spacy.load("en")

For safe measure, let's add some stopwords. It's a newspaper corpus, so it is likely we will be coming across variations of 'said' and 'Mister' which will not really add any value to the topic models.


In [5]:
my_stop_words = [u'say', u'\'s', u'mr', u'be', u'said', u'says', u'saying', 'today']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [6]:
doc = nlp(text.lower())

Voila! With the `English` pipeline, all the heavy lifting has been done. Let's see what went on under the hood.

In [7]:
doc

hundreds of people have been forced to vacate their homes in the southern highlands of new south wales as strong winds today pushed a huge bushfire towards the town of hill top. a new blaze near goulburn, south-west of sydney, has forced the closure of the hume highway. at about 4:00pm aedt, a marked deterioration in the weather as a storm cell moved east across the blue mountains forced authorities to make a decision to evacuate people from homes in outlying streets at hill top in the new south wales southern highlands. an estimated 500 residents have left their homes for nearby mittagong. the new south wales rural fire service says the weather conditions which caused the fire to burn in a finger formation have now eased and about 60 fire units in and around hill top are optimistic of defending all properties. as more than 100 blazes burn on new year's eve in new south wales, fire crews have been called to new fire at gunning, south of goulburn. while few details are available at this

## Computational Linguistics

Okay - now that we have our doc object, what exactly can we do with it?
We can see that the doc object now contains the entire corpus. This is important because we will be using this doc object to create our corpus for the machine learning algorithms. When creating a corpus for gensim/scikit-learn, we sometimes forget the incredible power which spaCy packs in its pipeline, so we will briefly demonstrate the same in this section with a smaller example sentence. Keep in mind that whatever we can do with a sentence, we can also just as well do with the entire corpus.

In [8]:
sent = nlp(u"Tom ran to the repair shop to fix his bicycle.")

Simple enough sentence, right? When we pass any kind of text through the spaCy pipeline, it becomes annotated. We will quickly have a look at the 3 most important of capabilities which spaCy provides - POS-tagging, NER-tagging, and dependency parsing.

#### POS-Tagging

In [9]:
for token in sent:
    print(token.text, token.pos_, token.tag_)

Tom PROPN NNP
ran VERB VBD
to ADP IN
the DET DT
repair NOUN NN
shop NOUN NN
to PART TO
fix VERB VB
his ADJ PRP$
bicycle NOUN NN
. PUNCT .


#### NER-Tagging

In [10]:
for token in sent:
    print(token.text, token.ent_type_)

Tom PERSON
ran 
to 
the 
repair 
shop 
to 
fix 
his 
bicycle 
. 


In [11]:
for ent in sent.ents:
    print(ent.text, ent.label_)

Tom PERSON


In [12]:
displacy.render(sent, style='ent', jupyter=True)

#### Dependency Parsing

In [13]:
for chunk in sent.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text)


Tom Tom nsubj ran
the repair shop shop pobj to
his bicycle bicycle dobj fix


In [14]:
for token in sent:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])


Tom nsubj ran VERB []
ran ROOT ran VERB [Tom, to, fix, .]
to prep ran VERB [shop]
the det shop NOUN []
repair compound shop NOUN []
shop pobj to ADP [the, repair]
to aux fix VERB []
fix advcl ran VERB [to, bicycle]
his poss bicycle NOUN []
bicycle dobj fix VERB [his]
. punct ran VERB []


In [15]:
displacy.render(sent, style='dep', jupyter=True)

/usr/local/Cellar/python3/3.5.2/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)


This is just an example of the kind of annotations spaCy adds when it runs any text through its pipeline. We will see in the very next section that spaCy has a bunch of other information as well, such as whether a token is a number or not, stop-word or not, and other information which comes in very handy when pre-processing text. 

## Continuing Cleaning

Have a quick look at the output of the doc object. It seems like nothing, right? But spaCy's internal data structure has done all the work for us. Let's see how we can create our corpus. You can check out what a gensim corpus looks like [here](https://radimrehurek.com/gensim/tut1.html).

In [16]:
# we add some words to the stop word list
texts, article = [], []
for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and w.text != 'I':
        # we add the lematized version of the word
        article.append(w.lemma_)
    # if it's a new line, it means we're onto our next document
    if w.text == '\n':
        texts.append(article)
        article = []

In [17]:
texts

[['hundred',
  'people',
  'force',
  'vacate',
  'home',
  'southern',
  'highland',
  'new',
  'south',
  'wale',
  'strong',
  'wind',
  'push',
  'huge',
  'bushfire',
  'town',
  'hill',
  'new',
  'blaze',
  'near',
  'goulburn',
  'south',
  'west',
  'sydney',
  'force',
  'closure',
  'hume',
  'highway',
  '4:00pm',
  'aedt',
  'mark',
  'deterioration',
  'weather',
  'storm',
  'cell',
  'move',
  'east',
  'blue',
  'mountain',
  'force',
  'authority',
  'decision',
  'evacuate',
  'people',
  'home',
  'outlying',
  'street',
  'hill',
  'new',
  'south',
  'wale',
  'southern',
  'highland',
  'estimate',
  'resident',
  'leave',
  'home',
  'nearby',
  'mittagong',
  'new',
  'south',
  'wale',
  'rural',
  'fire',
  'service',
  'weather',
  'condition',
  'cause',
  'fire',
  'burn',
  'finger',
  'formation',
  'ease',
  'fire',
  'unit',
  'hill',
  'optimistic',
  'defend',
  'property',
  'blaze',
  'burn',
  'new',
  'year',
  'eve',
  'new',
  'south',
  'wale'

And this is the magic of spaCy - just like that, we've managed to get rid of stopwords, punctauation markers, and added the lemmatized word. 

Sometimes topic models make more sense when 'New' and 'York' are treated as 'New_York' - we can do this by creating a bigram model and modifying our corpus accordingly.

In [18]:
bigram = gensim.models.Phrases(texts)

In [19]:
texts = [bigram[line] for line in texts]

In [20]:
texts[10]

['work',
 'continue',
 'morning',
 'restore',
 'power',
 'supply',
 'ten',
 'thousand',
 'home',
 'black',
 'wild',
 'storm',
 'strike',
 'south',
 'east',
 'queensland',
 'night',
 'gale',
 'force',
 'wind',
 'uproot',
 'tree',
 'bring',
 'power',
 'line',
 'damage',
 'home',
 'car',
 'energex',
 'ergon',
 'energy',
 'available',
 'person',
 'work',
 'night',
 'restore',
 'power',
 'location',
 'brisbane',
 'west',
 'toowoomba',
 'north',
 'sunshine',
 'coast',
 'boonah',
 'south',
 'west',
 'brisbane',
 'protective',
 'tarpaulin',
 'rip',
 'home',
 'undergo',
 'repair',
 'follow',
 'severe',
 'storm',
 'christma',
 'nambour',
 'people',
 'rescue',
 'high',
 'voltage',
 'power',
 'line',
 'fall',
 'car',
 'trap',
 'inside',
 'landsborough',
 'fierce',
 'wind',
 'send',
 'large',
 'tree',
 'crash',
 'house',
 'injure']

In [21]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [22]:
corpus[100]

[(76, 2),
 (185, 1),
 (222, 1),
 (275, 1),
 (288, 1),
 (423, 1),
 (476, 1),
 (541, 1),
 (609, 1),
 (710, 1),
 (846, 1),
 (977, 2),
 (1146, 1),
 (1152, 1),
 (1326, 2),
 (1351, 1),
 (1552, 1),
 (1562, 1),
 (1576, 1),
 (1584, 1),
 (1673, 3),
 (1856, 1),
 (1889, 2),
 (1950, 1),
 (1962, 1),
 (1968, 1),
 (1984, 1),
 (2604, 2),
 (2619, 1),
 (2713, 3),
 (2796, 1),
 (2941, 1),
 (2942, 1),
 (2943, 1),
 (2944, 1),
 (2945, 1),
 (2946, 1),
 (2947, 1),
 (2948, 1),
 (2949, 1),
 (2950, 1),
 (2951, 1),
 (2952, 1),
 (2953, 1),
 (2954, 1),
 (2955, 1),
 (2956, 1),
 (2957, 1),
 (2958, 1),
 (2959, 1),
 (2960, 1),
 (2961, 1),
 (2962, 1),
 (2963, 1),
 (2964, 1)]

## Topic Modelling

Topic Modelling refers to the probabilistic modelling of text documents as topics. Gensim remains the most popular library to perform such modelling, and we will be using it to perform our topic modelling. 

LDA, or Latent Dirichlet Allocation is arguably the most famous topic modelling algorithm out there. Out here we create a simple topic model with 10 topics. This is where the corpus we created earlier will come in handy.

In [23]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [24]:
ldamodel.show_topics()

[(0,
  '0.008*"australia" + 0.005*"year" + 0.005*"australian" + 0.004*"day" + 0.004*"new" + 0.003*"fire" + 0.003*"people" + 0.003*"wicket" + 0.003*"$" + 0.003*"police"'),
 (1,
  '0.005*"metre" + 0.004*"day" + 0.004*"union" + 0.004*"win" + 0.003*"australia" + 0.003*"meet" + 0.003*"tell" + 0.003*"al_qaeda" + 0.003*"official" + 0.003*"event"'),
 (2,
  '0.007*"australian" + 0.006*"report" + 0.006*"force" + 0.005*"area" + 0.005*"australia" + 0.005*"people" + 0.005*"al_qaeda" + 0.004*"unite_state" + 0.004*"afghanistan" + 0.004*"know"'),
 (3,
  '0.008*"australia" + 0.007*"australian" + 0.007*"government" + 0.005*"year" + 0.005*"people" + 0.005*"man" + 0.005*"day" + 0.003*"police" + 0.003*"claim" + 0.003*"power"'),
 (4,
  '0.005*"australian" + 0.005*"australia" + 0.004*"fire" + 0.004*"government" + 0.004*"arafat" + 0.004*"sydney" + 0.004*"year" + 0.004*"israeli" + 0.004*"people" + 0.003*"palestinian"'),
 (5,
  '0.005*"kill" + 0.005*"people" + 0.004*"palestinian" + 0.004*"israeli" + 0.004*"poli

This is a great way to get a view of what words end up appearing in our documents, and what kind of document topics might be present. For more details, such as the other topic models which Gensim provides, as well as ways to measure topic coherence (performance), and visualisation, the topic modelling notebook in the same directory will serve as a good resource.

## Text Classification

In the previous example, we worked with unlabelled, unstructured data. Classification is a machine learning task which is quite different from the previous examples because we are dealing with labelled data, and we know what classes we want to put our documents into - we are not discovering topics or classes.

For such an example, we would need to use a labelled data-set, and in our case we will be using the previously mentioned 20NG dataset.

In [25]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [26]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                             shuffle=True, random_state=42)
n_components = 5
labels = data_train.target
true_k = np.unique(labels).shape[0]

# convert to TF-IDF format
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', use_idf=True)
X_train = vectorizer.fit_transform(data_train.data)

# Reduce dimensions
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X_train = lsa.fit_transform(X_train)

In [27]:
# order of labels in `target_names` can be different from `categories`
data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42)

target_names = data_train.target_names
# split a training set and a test set
y_train, y_test = data_train.target, data_test.target

print("Extracting features from the test data using the same vectorizer")
X_test = vectorizer.transform(data_test.data)
X_test = lsa.fit_transform(X_test)

Extracting features from the test data using the same vectorizer


Take a minute to note the pre-processing steps we used above - it is less transparent than our method with spaCy, but it is still important to know and to be able to use the scikit-learn modules for the same. 

In [28]:
from sklearn.naive_bayes import GaussianNB

In [29]:
gnb = GaussianNB()
y_pred_NB = gnb.fit(X_train, y_train).predict(X_test)

In [30]:
from sklearn.svm import SVC

In [31]:
svm = SVC()
y_pred_SVM = svm.fit(X_train, y_train).predict(X_test) 

## Deep Learning

The final bit of our tutorial will explore the ideas of neural networks, and using RNNs to generate text.

A Recurrent Neural Network does one step better than other neural networks because of its ability to remember context, as each layer in the network is built with information from the previous layer - this additional context allows it to perform better. We will be using a particular variant of an RNN called LSTM, or Long Short Term Memory - as the name suggests, it has the ability to have short-term memory which can last for a long period of time. Whenever there is a significant time-lag between inputs, LSTMs tend to perform well - considering the nature of language, where a word which appears later on in a sentence is influenced by the context of the sentence, this property starts becoming more important. For a more detailed explanation on the mathematics or intuition behind an LSTM and RNN, the following blog posts can be very useful:

[Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
[Unreasonable Effectiveness of Reccurent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

For this part of the tutorial, we will be using the code written by my good friend Kirit Thadaka - you can find the original code over on this [GitHub repository](https://github.com/kirit93/Personal/blob/master/text_generation_keras/text_generation.ipynb).

In [32]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
SEQ_LENGTH = 100

In [33]:
test_x = np.array([1, 2, 0, 4, 3, 7, 10])
# one hot encoding
test_y = np_utils.to_categorical(test_x)
print(test_x)
print(test_y)

[ 1  2  0  4  3  7 10]
[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [34]:
# Using keras functional model
def create_functional_model(n_layers, input_shape, hidden_dim, n_out, **kwargs):
    drop        = kwargs.get('drop_rate', 0.2)
    activ       = kwargs.get('activation', 'softmax')
    mode        = kwargs.get('mode', 'train')
    hidden_dim  = int(hidden_dim)

    inputs      = Input(shape = (input_shape[1], input_shape[2]))
    model       = LSTM(hidden_dim, return_sequences = True)(inputs)
    model       = Dropout(drop)(model)
    model       = Dense(n_out)(model)


In [35]:
# Using keras sequential model
def create_model(n_layers, input_shape, hidden_dim, n_out, **kwargs):
    drop        = kwargs.get('drop_rate', 0.2)
    activ       = kwargs.get('activation', 'softmax')
    mode        = kwargs.get('mode', 'train')
    hidden_dim  = int(hidden_dim)
    model       = Sequential()
    flag        = True 

    if n_layers == 1:   
        model.add( LSTM(hidden_dim, input_shape = (input_shape[1], input_shape[2])) )
        if mode == 'train':
            model.add( Dropout(drop) )

    else:
        model.add( LSTM(hidden_dim, input_shape = (input_shape[1], input_shape[2]), return_sequences = True) )
        if mode == 'train':
            model.add( Dropout(drop) )
        for i in range(n_layers - 2):
            model.add( LSTM(hidden_dim, return_sequences = True) )
            if mode == 'train':
                model.add( Dropout(drop) )
        model.add( LSTM(hidden_dim) )

    model.add( Dense(n_out, activation = activ) )

    return model

In [36]:
def train(model, X, Y, n_epochs, b_size, vocab_size, **kwargs):    
    loss            = kwargs.get('loss', 'categorical_crossentropy')
    opt             = kwargs.get('optimizer', 'adam')
    
    model.compile(loss = loss, optimizer = opt)

    filepath        = "Weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint      = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
    callbacks_list  = [checkpoint]
    X               = X / float(vocab_size)
    model.fit(X, Y, epochs = n_epochs, batch_size = b_size, callbacks = callbacks_list)

The fit function will run the input batchwase n_epochs number of times and it will save the weights to a file whenever there is an improvement. This is taken care of through the callback. 

After the training is done or once you find a loss that you are happy with, you can test how well the model generates text.

In [37]:
def generate_text(model, X, filename, ix_to_char, vocab_size):
    
    # Load the weights from the epoch with the least loss
    model.load_weights(filename)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

    start   = np.random.randint(0, len(X) - 1)
    pattern = np.ravel(X[start]).tolist()

    # We seed the model with a random sequence of 100 so it can start predicting
    print ("Seed:")
    print ("\"", ''.join([ix_to_char[value] for value in pattern]), "\"")
    output = []
    for i in range(250):
        x           = np.reshape(pattern, (1, len(pattern), 1))
        x           = x / float(vocab_size)
        prediction  = model.predict(x, verbose = 0)
        index       = np.argmax(prediction)
        result      = index
        output.append(result)
        pattern.append(index)
        pattern = pattern[1 : len(pattern)]

    print("Predictions")
    print ("\"", ''.join([ix_to_char[value] for value in output]), "\"")

In [38]:
# filename    = 'data/game_of_thrones.txt'
# data        = open(filename).read()
# data        = data.lower()
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
data = open(lee_train_file).read()
# Find all the unique characters
chars       = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
ix_to_char  = dict((i, c) for i, c in enumerate(chars))
vocab_size  = len(chars)

print("List of unique characters : \n", chars)

print("Number of unique characters : \n", vocab_size)

print("Character to integer mapping : \n", char_to_int)

List of unique characters : 
 ['\n', ' ', '"', '$', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of unique characters : 
 81
Character to integer mapping : 
 {'1': 13, '<': 24, 't': 74, 'D': 30, 'P': 42, 'e': 59, '.': 10, '?': 26, '8': 20, 'M': 39, '2': 14, 'T': 46, 'o': 69, 'l': 66, 'f': 60, '\n': 0, '6': 18, 'm': 67, 'z': 80, 'n': 68, 'y': 79, "'": 5, 's': 73, ':': 22, 'S': 45, 'C': 29, 'I': 35, 'j': 64, '3': 15, 'k': 65, 'Z': 52, 'c': 57, 'v': 76, 'p': 70, ' ': 1, ';': 23, 'O': 41, 'A': 27, 'V': 48, 'X': 50, 'H': 34, '0': 12, 'g': 61, ',': 8, '$': 3, '[': 53, 'w': 77, 'h': 62, 'u': 75, 'R': 44, 'Q': 43, '&': 4, '4': 16, 'W': 49, ']': 54, '-':

In [39]:
list_X      = []
list_Y      = []

# Python append is faster than numpy append. Try it!
for i in range(0, len(data) - SEQ_LENGTH, 1):
    seq_in  = data[i : i + SEQ_LENGTH]
    seq_out = data[i + SEQ_LENGTH]
    list_X.append([char_to_int[char] for char in seq_in])
    list_Y.append(char_to_int[seq_out])

n_patterns  = len(list_X)

In [40]:
X           = np.reshape(list_X, (n_patterns, SEQ_LENGTH, 1)) # (n, 100, 1)
# Encode output as one-hot vector
Y           = np_utils.to_categorical(list_Y)

In [41]:
model   = create_model(1, X.shape, 256, Y.shape[1], mode = 'train')

In [42]:
train(model, X[:1024], Y[:1024], 2, 512, vocab_size)


Epoch 1/2
1024/1024 [==============================] - 6s 6ms/step - loss: 4.3684

Epoch 00001: loss improved from inf to 4.36838, saving model to Weights/weights-improvement-01-4.3684.hdf5
Epoch 2/2
1024/1024 [==============================] - 6s 6ms/step - loss: 4.2835

Epoch 00002: loss improved from 4.36838 to 4.28353, saving model to Weights/weights-improvement-02-4.2835.hdf5


In [43]:
generate_text(model, X, "Weights/weights-improvement-01-4.3050.hdf5", ix_to_char, vocab_size)

Seed:
" hat is all that we had hoped for." The video tape showed bin Laden speaking to supporters in a room, "
Predictions
"                                                                                                                                                                                                                                                            "


In [44]:
generate_text(model, X, "Weights/weights-improvement-02-4.0229.hdf5", ix_to_char, vocab_size)


Seed:
"  fell for 277. When play was abandoned due to rain a few overs short of scheduled stumps yesterday,  "
Predictions
"                                                                                                                                                                                                                                                            "
